# Lab 2: Health Check Function Tool

In this lab, you will learn how to integrate an Azure Function (health check) as a tool for your AI agent.

## Learning Objectives

- Understand how tools extend agent capabilities
- Create and register an Azure Function tool
- Call Azure Functions from the agent
- Handle tool responses

## Prerequisites

- Completed Lab 1: Create an Agent
- Azure Functions deployed (see [Functions Setup Guide](../docs/functions_setup.md))
- `.env` file configured with `AZURE_FUNCTION_APP_URL` and `AZURE_FUNCTION_KEY`

## Step 1: Import Required Modules

In [ ]:
from northwestern_foundry_agent import Settings, FoundryAgent
from northwestern_foundry_agent.foundry.tools import create_health_check_tool
from northwestern_foundry_agent.integrations.azure_functions import (
    AzureFunctionsClient,
    create_function_handlers,
)
from northwestern_foundry_agent.utils import setup_logging, get_logger

# Set up logging
setup_logging(Settings(log_level="INFO", log_format="text"))
logger = get_logger(__name__)

print("Modules imported successfully!")

## Step 2: Load Configuration and Verify

In [ ]:
settings = Settings()

print(f"Azure Functions URL: {settings.azure_function_app_url or 'Not configured'}")
print(f"Function Key configured: {'Yes' if settings.azure_function_key else 'No'}")

if not settings.functions_configured:
    print("\n⚠️ Warning: Azure Functions not configured.")
    print("Update .env with AZURE_FUNCTION_APP_URL and AZURE_FUNCTION_KEY")

## Step 3: Test the Azure Function Directly

Before integrating with the agent, let's test the Azure Function directly using the client.

In [ ]:
# Test the health check function directly
if settings.functions_configured:
    try:
        client = AzureFunctionsClient(settings)
        health = await client.health_check()
        
        print("Health Check Response:")
        print(f"  Status: {health.status}")
        print(f"  Service: {health.service_name}")
        print(f"  Version: {health.version}")
        print(f"  Healthy: {health.is_healthy}")
    except Exception as e:
        print(f"Error calling health check: {e}")
else:
    print("Skipping direct test (functions not configured)")
    print("\nExpected response:")
    print("  Status: healthy")
    print("  Service: northwestern-foundry-functions")
    print("  Version: 1.0.0")

## Step 4: Create the Tool Definition

Now let's create a tool definition that tells the agent about the health check function.

In [ ]:
# Create the health check tool
health_endpoint = f"{settings.azure_function_app_url}/api/health" if settings.functions_configured else "https://example.azurewebsites.net/api/health"
health_tool = create_health_check_tool(health_endpoint)

print("Tool Definition:")
print(f"  Name: {health_tool.name}")
print(f"  Description: {health_tool.description}")
print(f"  Endpoint: {health_tool.endpoint}")
print(f"  HTTP Method: {health_tool.http_method}")

# View the function definition format
print("\nFunction Definition (for AI):")
import json
print(json.dumps(health_tool.to_function_definition(), indent=2))

## Step 5: Create Agent with Health Check Tool

Now let's create an agent and register the health check tool.

In [ ]:
AGENT_INSTRUCTIONS = """
You are a helpful assistant that can monitor system health.

When asked about system status or health:
1. Use the health_check tool to get current status
2. Report the status clearly to the user
3. Highlight any issues if the system is not healthy

Be concise and informative in your responses.
"""

# Create agent and register tool
if settings.is_configured and settings.functions_configured:
    try:
        agent = FoundryAgent(settings)
        
        # Create function client and handlers
        func_client = AzureFunctionsClient(settings)
        handlers = create_function_handlers(func_client)
        
        # Register the health check tool
        agent.register_tool(health_tool, handlers["health_check"])
        
        # Create the agent in Azure
        created_agent = agent.create_agent(
            name="health-monitor-agent",
            instructions=AGENT_INSTRUCTIONS,
        )
        
        print(f"Agent created with health check tool!")
        print(f"Agent ID: {created_agent.id}")
        print(f"Registered tools: {list(agent.tools.keys())}")
    except Exception as e:
        print(f"Error: {e}")
        agent = None
else:
    print("Skipping (Azure not fully configured)")
    agent = None

## Step 6: Test the Agent with Health Check

Let's ask the agent about system health - it should use the tool automatically.

In [ ]:
if agent and agent.agent:
    try:
        response = agent.run("What is the current system health status?")
        print("Agent Response:")
        print("-" * 50)
        print(response)
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Agent not available. Example interaction:")
    print("-" * 50)
    print("User: What is the current system health status?")
    print("")
    print("[Agent calls health_check tool]")
    print("")
    print("Agent: The system is currently healthy. Here are the details:")
    print("- Status: healthy")
    print("- Service: northwestern-foundry-functions")
    print("- Version: 1.0.0")

## Step 7: Clean Up

In [ ]:
if agent and agent.agent:
    try:
        agent.delete_agent()
        print("Agent deleted successfully!")
    except Exception as e:
        print(f"Error: {e}")
else:
    print("No agent to delete.")

## Summary

In this lab, you learned how to:

1. ✅ Test Azure Functions directly using the client
2. ✅ Create a tool definition for the health check function
3. ✅ Register tools with the Foundry agent
4. ✅ Let the agent automatically call tools based on user queries

## Key Concepts

- **Tool Definition**: Describes what the tool does and its parameters
- **Tool Handler**: Function that actually executes when the tool is called
- **Function Calling**: The agent decides when to use tools based on the conversation

## Next Steps

Continue to **Lab 3: Quote Function Tool** to add another function and see how agents handle multiple tools.